# ChatFlow Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json, copy

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_threads, run_dispatch_worker_thread
from aiflows import flow_verse

dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "coflows"},
]
flow_verse.sync_dependencies(dependencies)

[2024-03-17 07:25:50,403][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/ChatFlow-languages/flow_modules...
[2024-03-17 07:25:50,544][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-17 07:25:50,545][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/ChatFlow-languages/flow_modules/aiflows/ChatFlowModule']

# Connect to CoLink Server

In [3]:
cl = colink_utils.start_colink_server()

# Start a few default workers

In [4]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_endpoint: Chatbot
flow_id: d5df4e44-7f15-47f3-aa0a-4c4ac999d375
owner_id: local
message_paths: ['push_tasks:7e5080fe-857e-4fa6-ae76-3ccf733c992c:msg']
parallel_dispatch: False

Input message source: Proxy_Chatbot

~~~ Dispatch task ~~~
flow_endpoint: Chatbot
flow_id: ad5feb52-5c2b-411c-9202-257a3b98d438
owner_id: local
message_paths: ['push_tasks:085b2bf6-009c-4e83-9b7b-45d69f936c49:msg']
parallel_dispatch: False

Input message source: Proxy_Chatbot


In [5]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_endpoint: Chatbot
flow_id: d5df4e44-7f15-47f3-aa0a-4c4ac999d375
owner_id: local
message_paths: ['push_tasks:40c9884c-89dd-471b-96e1-dd393bccfc37:msg']
parallel_dispatch: False

Input message source: Proxy_Chatbot

~~~ Dispatch task ~~~
flow_endpoint: Chatbot
flow_id: ad5feb52-5c2b-411c-9202-257a3b98d438
owner_id: local
message_paths: ['push_tasks:07e477b0-f653-476a-807c-dadb29575554:msg']
parallel_dispatch: False

Input message source: Proxy_Chatbot


# Serve ChatFlow

In [6]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow",
    flow_endpoint="Chatbot",
)

Started serving flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow at flows:Chatbot.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

## Get two instances with different configurations

In [7]:
root_dir = "."
cfg_path = os.path.join(root_dir, "flow_modules/aiflows/ChatFlowModule/demo.yaml")
cfg = read_yaml_file(cfg_path)

In [8]:
args_A = copy.deepcopy(cfg)
args_A["system_message_prompt_template"]["template"] = \
"You are a helpful chatbot that truthfully answers questions. Answer in the language English."

chatbot_A = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chatbot",
    config_overrides=args_A,
)

Mounted d5df4e44-7f15-47f3-aa0a-4c4ac999d375 at flows:Chatbot:mounts:local:d5df4e44-7f15-47f3-aa0a-4c4ac999d375


In [9]:
args_B = copy.deepcopy(cfg)
args_B["system_message_prompt_template"]["template"] = \
"You are a helpful chatbot that truthfully answers questions. Answer in the language Croatian."


chatbot_B = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="Chatbot",
    config_overrides=args_B,
)

Mounted ad5feb52-5c2b-411c-9202-257a3b98d438 at flows:Chatbot:mounts:local:ad5feb52-5c2b-411c-9202-257a3b98d438


## Conversation with chatbot A

In [10]:
input_message = FlowMessage(
    data={"id": 0, "question": "What is the capital of Switzerland?"},
)

future = chatbot_A.get_reply_future(input_message)
print(future.get_data())

input_message = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

future = chatbot_A.get_reply_future(input_message)
print(future.get_data())

{'api_output': 'The capital of Switzerland is Bern.'}
{'api_output': 'Bern, the capital of Switzerland, is located in the central part of the country, on the Swiss Plateau.'}


## Conversation with chatbot B

In [11]:
input_message = FlowMessage(
    data={"id": 0, "question": "What is the capital of Switzerland?"},
)

future = chatbot_B.get_reply_future(input_message)
print(future.get_data())

input_message = FlowMessage(
    data={"id": 0, "query": "Where is it located?"},
)

future = chatbot_B.get_reply_future(input_message)
print(future.get_data())

{'api_output': 'Glavni grad Švicarske je Bern.'}
{'api_output': 'Bern se nalazi u središnjem dijelu Švicarske, na rijeci Aare.'}
